In [1]:
from typing import List, Callable


def validate_extended_matrix(matrix: List[List[float]], n: int) -> None:
    extended_length = n + 1
    for i in range(n):
        if len(matrix[i]) != extended_length:
            raise Exception(f'Incorrect matrix was passed. Expected length {extended_length} on row {i}.')


def copy_matrix(matrix: List[List[float]]) -> List[List[float]]:
    return [[col for col in row] for row in matrix]


def solve_gauss(original_matrix: List[List[float]]) -> tuple[List[float], List[List[float]]]:
    n = len(original_matrix)
    validate_extended_matrix(original_matrix, n)
    operative_matrix = copy_matrix(original_matrix)

    for i in range(n):
        for j in range(i + 1, n):
            row_multiplier = operative_matrix[j][i] / operative_matrix[i][i]
            for k in range(i, n + 1):
                operative_matrix[j][k] -= row_multiplier * operative_matrix[i][k]

    ans = [0 for _ in range(n)]
    for i in reversed(range(n)):
        ans_numerator = operative_matrix[i][n]
        for j in range(i + 1, n):
            ans_numerator -= operative_matrix[i][j] * ans[j]

        ans[i] = ans_numerator / operative_matrix[i][i]

    return ans, operative_matrix


def solve_gauss_jordan(original_matrix: List[List[float]]) -> tuple[List[float], List[List[float]]]:
    n = len(original_matrix)
    validate_extended_matrix(original_matrix, n)
    operative_matrix = copy_matrix(original_matrix)

    for i in range(n):
        for j in filter(lambda arg1: i != arg1, range(n)):
            row_multiplier = operative_matrix[j][i] / operative_matrix[i][i]
            for k in range(i, n + 1):
                operative_matrix[j][k] -= row_multiplier * operative_matrix[i][k]

    ans = [row[n] / row[i] for i, row in enumerate(operative_matrix)]
    return ans, operative_matrix


def solve_jacobi(original_matrix: List[List[float]], initial_vector: List[float], iterations: int) -> List[float]:
    n = len(original_matrix)
    validate_extended_matrix(original_matrix, n)

    if len(initial_vector) != n:
        raise Exception(f'The vector of initial approximation should have length of {n}')

    last_approx = initial_vector

    for i in range(iterations):
        current_approx = [0 for _ in range(n)]

        for j in range(n):
            for k in range(n):
                if j == k:
                    continue

                current_approx[j] -= (original_matrix[j][k] / original_matrix[j][j]) * last_approx[k]

            current_approx[j] += original_matrix[j][n] / original_matrix[j][j]

        last_approx = current_approx

    return last_approx


def solve_gauss_seidel(original_matrix: List[List[float]], initial_vector: List[float], iterations: int) -> List[float]:
    n = len(original_matrix)
    validate_extended_matrix(original_matrix, n)

    if len(initial_vector) != n:
        raise Exception(f'The vector of initial approximation should have length of {n}')

    last_approx = initial_vector

    for i in range(iterations):
        current_approx = [0 for _ in range(n)]

        for j in range(n):
            for k in range(n):
                if j == k:
                    continue

                multiplier = current_approx[k] if k < j else last_approx[k]
                current_approx[j] -= (original_matrix[j][k] / original_matrix[j][j]) * multiplier

            current_approx[j] += original_matrix[j][n] / original_matrix[j][j]

        last_approx = current_approx

    return last_approx


def wrap_iterative_method(original_matrix: List[List[float]], func: Callable[[List[List[float]], List[float], int], List[float]]) -> tuple[List[float], List[List[float]] | None]:
    wrapped_ans = func(original_matrix, [0 for _ in original_matrix], 100)
    return wrapped_ans, None


problems = [
    [[9, -6, -2, 10], [-2, 8, -3, 0], [-1, -4, 6, 0]],
    [[-1, -1, 6, 2], [-1, 6, -1, 34], [6, -1, -1, 12]],
    [[1, 4, -3, 4], [4, 20, 14, 20], [3, 14, 14, 14]],
    [[4, 1, 2, 6], [1, -3, -1, 3], [3, 1, 5, 10]]
]

solving_funcs = [(solve_gauss, 'Gauss'), (solve_gauss_jordan, 'Gauss-Jordan'),
                 (lambda arg1: wrap_iterative_method(arg1, solve_jacobi), 'Jacobi x100'),
                 (lambda arg1: wrap_iterative_method(arg1, solve_gauss_seidel), 'Gauss-Seidel x100')]


def print_info(ans: tuple[List[float], List[List[float]] | None], original_matrix: List[List[float]], func_name: str) -> None:
    print(func_name)
    print(f'Original matrix: {original_matrix}')

    if len(ans) > 1 and ans[1] is not None:
        print(f'Matrix after transformations ({func_name}): {ans[1]}')

    print(f'Result: {ans[0]}')
    print()


def solve() -> None:
    for func, func_name in solving_funcs:
        for problem_matrix in problems:
            ans = func(problem_matrix)
            print_info(ans, problem_matrix, func_name)


solve()

Gauss
Original matrix: [[9, -6, -2, 10], [-2, 8, -3, 0], [-1, -4, 6, 0]]
Matrix after transformations (Gauss): [[9, -6, -2, 10], [0.0, 6.666666666666667, -3.4444444444444446, 2.2222222222222223], [0.0, 0.0, 3.3666666666666663, 2.666666666666667]]
Result: [1.7821782178217822, 0.7425742574257427, 0.7920792079207922]

Gauss
Original matrix: [[-1, -1, 6, 2], [-1, 6, -1, 34], [6, -1, -1, 12]]
Matrix after transformations (Gauss): [[-1, -1, 6, 2], [0.0, 7.0, -7.0, 32.0], [0.0, 0.0, 28.0, 56.0]]
Result: [3.428571428571429, 6.571428571428571, 2.0]

Gauss
Original matrix: [[1, 4, -3, 4], [4, 20, 14, 20], [3, 14, 14, 14]]
Matrix after transformations (Gauss): [[1, 4, -3, 4], [0.0, 4.0, 26.0, 4.0], [0.0, 0.0, 10.0, 0.0]]
Result: [0.0, 1.0, 0.0]

Gauss
Original matrix: [[4, 1, 2, 6], [1, -3, -1, 3], [3, 1, 5, 10]]
Matrix after transformations (Gauss): [[4, 1, 2, 6], [0.0, -3.25, -1.5, 1.5], [0.0, 0.0, 3.3846153846153846, 5.615384615384615]]
Result: [0.9772727272727272, -1.227272727272727, 1.659090